In [1]:
import pandas as pd 
import numpy as np 
import plotly.graph_objects as go
import plotly.io as pio

In [ ]:
# df = pd.read_csv("/home/chemachiles/OneDrive/gamma/assets/gzr.out",sep="\s+") 
# nn = pd.read_csv("/home/chemachiles/OneDrive/gamma/assets/ngzr.out",sep="\s+") 
# cc = df.merge(nn, on='#Frame')
# cc = cc.set_index('#Frame')

df = pd.read_csv("/home/chemachiles/OneDrive/gamma/assets/gzr2.out",sep="\s+") 
nn = pd.read_csv("/home/chemachiles/OneDrive/gamma/assets/ngzr2.out",sep="\s+") 
cc = df.merge(nn, on='#Frame')
cc = cc.set_index('#Frame')

In [ ]:
#Definimos las subpartes de analisis de e2012  y de la cavidad
imidazol = ["@N2_", "@C10_", "@N3_", "@C23_", "@C22_","@H9_", "@H20_", "@C25_", "@H24_", "@H25_", "@H26_"]
metoxi = ["@C20_", "@C8_", "@C19_", "@C17_", "@C5_", "@C9_", "@O2_", "@C24_", "@H21_", "@H22_", "@H23_", "@H19_", "@H5_", "@H8_"]
piper = ["@C14_", "@C15_", "@N1_", "@C11_", "@C12_", "@C1_", "@O1_", "@H10_", "@H11_", "@H12_", "@H13_", "@H1_", "@H2_", "@C3_", "@H3_", "@C13_", "@H14_"]
metil = ["@C13_", "@H14_", "@C16_", "@H15_", "@H16_", "@H17_"]
# metil = ["@C16_", "@H15_", "@H16_", "@H17_"]
fluoro = ["@C2_", "@C4_", "@C6_", "@C21_", "@C7_", "@C18_", "@H4_", "@H6_", "@F1_", "@H7_", "@H18_"]
gzr = imidazol+metoxi+piper+metil+fluoro
partes_e2012 = [imidazol,metoxi,piper,metil,fluoro,gzr]


#Aminoacidos de la cavidad
aa = pd.read_csv("/home/chemachiles/OneDrive/gamma/assets/numerado.csv")    
aa["Subunidad"] = aa["Subunidad"].replace("SUS", "AB")
cav= ["209","210","700", "701", "705", "706","707","708","709","710","767","768", "771", "772","773","774", "775", "776", "779", "826","827","828","829","830","831","832","834", "835","1329","1330","1331","1334", "1338","1341", "1342","1343","1344","1345"]
cav_chido = [aa[(aa["Numeracion"]==int(i))&(aa["Sistema"]=="Complejo Enzima-Sustrato")].values[0][1] + "-" +str(aa[(aa["Numeracion"]==int(i))&(aa["Sistema"]=="Complejo Enzima-Sustrato")].values[0][2])+"-" +str(aa[(aa["Numeracion"]==int(i))&(aa["Sistema"]=="Complejo Enzima-Sustrato")].values[0][4])for i in  cav]

In [ ]:
#Primero vamos a separar el df de contactos en cada una de las subpartes de E2012 
cols = [[] for i in partes_e2012]
c = -1
for partes in partes_e2012: #para cada parte de E2012
    c+=1
    for atom in partes:
        for  col in cc.columns.values: 
            if atom in col:   #separa en una lista los nombres de las columnas 
                cols[c].append(col)

In [ ]:
#corrobora que hay el mismo numero de columnas en cols que en el df 
np.sum(np.array([len(i) for i in cols[:5]])), cc.shape[1]

In [ ]:
#Ahora vamos a separar por subpartes y por cada aminoacido, asi podemos ver con quien si esta interactuando
cols_amino = [[[] for i in cav] for j in partes_e2012]
c = -1
for partes in cols:               #Para cada parte en e2012
    c+=1
    c2=-1
    for aa in cav:                #checar cada aminoacido
        c2+=1
        for atom in partes:       #si hay un contacto o no 
            if aa in atom:
                cols_amino[c][c2].append(atom)

In [ ]:
#corrobora que hay el mismo numero de columnas en cols_amino que en el df 
tt = 0
for i in  range(len(cols_amino)-1):
    for j in range(len(cols_amino[i])):
        tt += cc[cols_amino[i][j]].shape[1]
        
tt, cc.shape[1]

In [ ]:
#Hagamos la sumas de los contactos
contactos = [[[] for i in cav] for j in partes_e2012]
c = -1
for i in  cols_amino:                #Para cada fragmento de E2012
    c+=1
    c2=-1
    for amino in i:                  #Que tiene asociado por cada aminoacido de la cavidad
        c2+=1
        if len(amino)==0:            #Si no hay columnas con contactos en automatico pon cero
            contactos[c][c2].append(0)
        else:
#             contactos[c][c2].append(np.mean(np.sum(cc[cols_amino[c][c2]].values, axis=1)/np.max(np.max(cc[cols_amino[c][c2]].values,axis=1))))
            contactos[c][c2].append(np.mean(np.sum(cc[cols_amino[c][c2]].values, axis=1)/np.max(np.sum(cc[cols_amino[c][c2]].values, axis=1))))
#La fraccion de contactos la calculo como contactos promedio por frame / el numero de contactos posibles  

In [ ]:
cav_chido[0] ="<b>NCST-ILE-242"
cav_chido[1] ="<b>NCST-ASN-243"
cav_chido[2] ="<b>PSEN-PHE-105"
cav_chido[3] ="<b>PSEN-TYR-106"
# cav_chido[12] ="<b>PSEN-PHE-176"
cav_chido[13] ="<b>PSEN-PHE-177"
cav_chido[16] ="<b>PSEN-ILE-180"
cav_chido[24] ="<b>PSEN-VAL-236"
cav_chido[27] ="<b>PSEN-TYR-240"

In [ ]:
cav[]

In [ ]:
contactos = np.array(contactos).reshape(6,len(cav_chido))

fig2 = go.Figure()
fig2.add_traces(go.Heatmap(z=contactos.flatten(),
                          x=cav_chido*6,
                          y=np.array([[i]*len(cav) for i in["Metilmidazol","Metoxifenilo","Piperidinona","Metilo","Fluorofenilo","E2012"]]).flatten(),
                          colorscale='OrRd'))

fig2.update_layout(title="Contactos a 5Å",title_x=0.5,font=dict(family="Courier New, monospace",size=45))
fig2.data[0].colorbar.title.text = "Frac. de contactos"
fig2.data[0].colorbar.titleside = "right"
fig2.update_xaxes(tickangle=-90,tickfont=dict(family="Courier New, monospace",size=30))
fig2.update_yaxes(tickfont=dict(family="Courier New, monospace",size=45))

In [ ]:
pio.write_image(fig2, "/home/chemachiles/OneDrive/gamma/figure/contactos_nuevasimul.png", width=3*600, height=1.5*600, scale=1)